# Endometriosis Classification System
This version of the expert system is the exact same as the one contained in classifier.py, except it contains a user interface (implemented as a prompt map) to obtain case facts from the user directly during runtime, through natural language. 

In [34]:
import clips
import pandas as pd
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [35]:
def count_symptoms(*args):
    count = 0
    for symptom in args:
        if symptom == "yes":
            count += 1
    return count

# create the CLIPS environment
env = clips.Environment()

# env.define_function(store_result)
env.define_function(count_symptoms)

# Templates

## Part 1: Defining Fact Templates (that are consistent with endometriosis)

In [36]:
# patient endometriosis symptoms
DEFTEMPLATE_PATIENT_ENDOMETRIOSIS_SYMPTOMS = """
(deftemplate patient_endo_symptoms
    (slot endometriosis (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot abdominal_pelvic_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot dysmenorrhea (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot pain_with_sex (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot dyschezia (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot dysuria (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot infertility (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot pelvic_perineal_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot adenomyosis (type SYMBOL)
        (allowed-symbols yes no unknown))

    )
"""
env.build(DEFTEMPLATE_PATIENT_ENDOMETRIOSIS_SYMPTOMS)

# patient concomitant disease symptoms
DEFTEMPLATE_PATIENT_CONCOMITANT_DISEASE_SYMPTOMS = """
(deftemplate patient_concomitant_disease_symptoms
    (slot amenorrhea (type SYMBOL) 
        (allowed-symbols yes no unknown))
    (slot constipation (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot diarrhea (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot flank_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot hematuria (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot frequent_urination (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot ibs (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot interstitial_cystitis (type SYMBOL)
        (allowed-symbols yes no unknown))

)
"""
env.build(DEFTEMPLATE_PATIENT_CONCOMITANT_DISEASE_SYMPTOMS)

# status of whether or not patient has symptoms consistent with endo
DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION = """
(deftemplate endometriosis_inclusion
    (slot meets_criteria (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION)

# status of whether or not patient has symptoms consistent with non-endo diseases
DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION = """
(deftemplate concomitant_disease_inclusion
    (slot meets_criteria (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION)

# status of patient being classified as case or control
DEFTEMPLATE_NEW_PHENOTYPE_STATUS = """
(deftemplate new_phenotype_status
    (slot endometriosis_phenotype_status (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_NEW_PHENOTYPE_STATUS)

# Initialize Knowledge Base

In [38]:
# Add deffacts that all statuses are unknown initially
DEFFACTS_INITIAL_STATUS = """
(deffacts starting_inclusion_exclusion_facts "Set the initial templates to unknown"
    (endometriosis_inclusion (meets_criteria unknown))
    (concomitant_disease_inclusion (meets_criteria unknown))
    (new_phenotype_status (endometriosis_phenotype_status unknown))
)
"""
env.build(DEFFACTS_INITIAL_STATUS)

env.reset()

# User Interface and Inference Rules

In [ ]:
# These are the prompts that will be displayed to the user when requesting various template:slot inputs.
prompt_map = {
    "patient_endo_symptoms:abdominal_pelvic_pain": "Does the patient's EHR contain the abominal pelvic pain ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:dysmenorrhea": "Does the patient's EHR contain the dysmenorrhea ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:pain_with_sex": "Does the patient's EHR contain the dyspareunia ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:dyschezia" : "Does the patient's EHR contain the dyschezia ICD code(yes or no or unknown)?: ",
    "patient_endo_symptoms:dysuria" : "Does the patient's EHR contain the dysuria ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:infertility" : "Does the patient's EHR contain the infertility ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:pelvic_perineal_pain" : "Does the patient's EHR contain the pelvic perineal pain ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:has_adenomyosis" : "Does the patient's EHR contain the adenomyosis ICD code (yes or no or unknown)?: ",
    "patient_endo_symptoms:endometriosis" : "Does the patient's EHR contain the endometriosis ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:amenorrhea" : "Does the patient's EHR contain the amenorrhea ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:constipation" : "Does the patient's EHR contain the constipation related to IBS ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:diarrhea" : "Does the patient's EHR contain the diarrhea ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:flank_pain" : "Does the patient's EHR contain the flank pain ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:hematuria" : "Does the patient's EHR contain the hematuria ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:frequent_urination" : "Does the patient's EHR contain the frequent urination ICD code (yes or no or unknonw)?: ", 
    "patient_concomitant_disease_symptoms:interstitial_cystitis" : "Does the patient's EHR contain the interstitial cystitis ICD code (yes or no or unknown)?: ",
    "patient_concomitant_disease_symptoms:ibs" : "Does the patient's EHR contain the irritable bowel syndrome ICD code (yes or no or unknown)?: "
}

DEFRULE_READ_INITIAL_INCLUSION = """
(defrule reading_input_initial
    =>
    (read_assert patient_endo_symptoms)
    (read_assert patient_concomitant_disease_symptoms)
)
"""
env.build(DEFRULE_READ_INITIAL_INCLUSION)

DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_NOT_MET = """
(defrule endo-inclusion-criteria-not-met "Rule to define a person not having any symptoms consistent with endometriosis"
    (logical
        (and
            (patient_endo_symptoms (abdominal_pelvic_pain ~yes))  
            (patient_endo_symptoms (dysmenorrhea ~yes))   
            (patient_endo_symptoms (pain_with_sex ~yes)) 
            (patient_endo_symptoms (dyschezia ~yes)) 
            (patient_endo_symptoms (dysuria ~yes)) 
            (patient_endo_symptoms (infertility ~yes))   
            (patient_endo_symptoms (pelvic_perineal_pain ~yes))
            (patient_endo_symptoms (adenomyosis ~yes))
            (patient_endo_symptoms (endometriosis ~yes))
        )
    )

    ?f1 <-(endometriosis_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria no))
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_NOT_MET)

# indicates presence of symptom(s) that are consistent with endometriosis
DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_MET = """
(defrule endo-inclusion-criteria-met "Rule to define a person as having symptom(s) consistent with endometriosis"
    (patient_endo_symptoms 
        (abdominal_pelvic_pain ?v1) 
        (dysmenorrhea ?v2) 
        (pain_with_sex ?v3) 
        (dyschezia ?v4) 
        (dysuria ?v5) 
        (infertility ?v6) 
        (pelvic_perineal_pain ?v7)
        (adenomyosis ?v8)
        (endometriosis ?v9))
    
    ?f1 <-(endometriosis_inclusion (meets_criteria unknown))
    
    => 
    (bind ?num_symptoms (count_symptoms ?v1 ?v2 ?v3 ?v4 ?v5 ?v6 ?v7 ?v8 ?v9))
    (if (>= ?num_symptoms 2) then
        (modify ?f1 (meets_criteria yes))
    )
    (if (< ?num_symptoms 2) then
        (modify ?f1 (meets_criteria no))
    )
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_MET)

# indicates presence of symptoms that indicate no concomitant disease (along with or instead of) endometriosis
DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_NOT_MET = """
(defrule concomitant-inclusion-criteria-not-met "Rule to define a person as having symptom(s) not consistent with other (non-endo) disease based on symptoms"
    (logical
        (and
            (patient_concomitant_disease_symptoms (amenorrhea ~yes))  
            (patient_concomitant_disease_symptoms (constipation ~yes))   
            (patient_concomitant_disease_symptoms (diarrhea ~yes)) 
            (patient_concomitant_disease_symptoms (flank_pain ~yes)) 
            (patient_concomitant_disease_symptoms (hematuria ~yes)) 
            (patient_concomitant_disease_symptoms (frequent_urination ~yes))
            (patient_concomitant_disease_symptoms (ibs ~yes))
            (patient_concomitant_disease_symptoms (interstitial_cystitis ~yes))
        )
    )

    ?f1 <-(concomitant_disease_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria no))
)
"""
env.build(DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_NOT_MET)

# indicates presence of symptoms that indicate concomitant disease (along with or instead of) endometriosis
DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_MET = """
(defrule concomitant-inclusion-criteria-met "Rule to define a person as having symptom(s) consistent with other (non-endo) disease based on inclusion criteria facts"
    (patient_concomitant_disease_symptoms 
        (amenorrhea ?v1) 
        (constipation ?v2) 
        (diarrhea ?v3) 
        (flank_pain ?v4) 
        (hematuria ?v5) 
        (frequent_urination ?v6)
        (ibs ?v7)
        (interstitial_cystitis ?v8)
    ) 
    
    ?f1 <-(concomitant_disease_inclusion  (meets_criteria unknown))
    
    => 
    (bind ?num_symptoms (count_symptoms ?v1 ?v2 ?v3 ?v4 ?v5 ?v6 ?v7 ?v8))
    (if (>= ?num_symptoms 2) then
        (modify ?f1 (meets_criteria yes))
    )
    (if (< ?num_symptoms 2) then
        (modify ?f1 (meets_criteria no))
    )

)
"""
env.build(DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_MET)
# not going to try and classify the concomitant disease - but will factor whether they do have a concomitant disease (IBS, or interstitial cystitis, or adenomyosis) in my evaluation
    # because I don't have detailed symptom data for all of the concomitant diseases, only some of them

# Reporting (Output) Rules

In [49]:
DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_INCLUSION = """
(defrule endo-and-concomitant-inclusion
    (endometriosis_inclusion (meets_criteria yes))
    (concomitant_disease_inclusion (meets_criteria yes))
    ?f1 <-(new_phenotype_status (endometriosis_phenotype_status unknown))
    =>
    (modify ?f1 (endometriosis_phenotype_status yes))
    (println "___________")
    (println "Patient has at least 1 symptom(s) that are consistent with both endometriosis and additional concomitant disease (irritable bowel syndrome, interstitial cystitis, urinary tract stones, or a reproductive tract anomaly). The purpose of this system is to identify endometriosis cases for large-scale research studies - this is NOT intended to be a formal diagnosis; classification was made based on limited symptoms without lab tests/physical exam results and further confirmation may be necessary.")
    (println "___________")
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_INCLUSION)

DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_EXCLUSION = """
(defrule endo-and-concomitant-exclusion
    (endometriosis_inclusion (meets_criteria no))
    (concomitant_disease_inclusion (meets_criteria no))
    ?f1 <-(new_phenotype_status (endometriosis_phenotype_status unknown))
    =>
    (modify ?f1 (endometriosis_phenotype_status no))
    (println "___________")
    (println "Patient has no symptoms that are consistent with endometriosis or other phenotypes that we screened for (irritable bowel syndrome, interstitial cystitis, urinary tract stones, or a reproductive tract anomaly). The purpose of this system is to identify endometriosis cases for large-scale research studies - this is NOT intended to be a formal diagnosis; classification was made based on limited symptoms without lab tests/physical exam results and further confirmation may be necessary.")
    (println "___________")
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_EXCLUSION)

DEFRULE_ONLY_ENDOMETRIOSIS_INCLUSION = """
(defrule only-endo-inclusion
    (endometriosis_inclusion (meets_criteria yes))
    (concomitant_disease_inclusion (meets_criteria no))
    ?f1 <-(new_phenotype_status (endometriosis_phenotype_status unknown))
    =>
    (modify ?f1 (endometriosis_phenotype_status yes))
    (println "___________")
    (println "Patient has at least 1 symptom that is consistent only with endometriosis and no symptoms consistent with additional diseases (irritable bowel syndrome, interstitial cystitis, urinary tract stones, or a reproductive tract anomaly). The purpose of this system is to identify endometriosis cases for large-scale research studies - this is NOT intended to be a formal diagnosis; classification was made based on limited symptoms without lab tests/physical exam results and further confirmation may be necessary.")
    (println "___________")
)
"""
env.build(DEFRULE_ONLY_ENDOMETRIOSIS_INCLUSION)

DEFRULE_ONLY_ENDOMETRIOSIS_EXCLUSION = """
(defrule only-endo-exclusion
    (endometriosis_inclusion (meets_criteria no))
    (concomitant_disease_inclusion (meets_criteria yes))
    ?f1 <-(new_phenotype_status (endometriosis_phenotype_status unknown))
    =>
    (modify ?f1 (endometriosis_phenotype_status no))
    (println "___________")
    (println "Patient has at least 1 symptom that is consistent only with non-Endometriosis phenotypes (irritable bowel syndrome, interstitial cystitis, urinary tract stones, or a reproductive tract anomaly). The purpose of this system is to identify endometriosis cases for large-scale research studies - this is NOT intended to be a formal diagnosis; classification was made based on limited symptoms without lab tests/physical exam results and further confirmation may be necessary.")
    (println "___________")
)
"""
env.build(DEFRULE_ONLY_ENDOMETRIOSIS_EXCLUSION)

# Execution

In [50]:
env.reset();
env.run();

___________
Patient has at least 1 symptom(s) that are consistent with both endometriosis and additional concomitant disease (irritable bowel syndrome, interstitial cystitis, urinary tract stones, or a reproductive tract anomaly). The purpose of this system is to identify endometriosis cases for large-scale research studies - this is NOT intended to be a formal diagnosis; classification was made based on limited symptoms without lab tests/physical exam results and further confirmation may be necessary.
___________


In [51]:
print_facts(env)

(endometriosis_inclusion (meets_criteria yes))
(concomitant_disease_inclusion (meets_criteria yes))
(new_phenotype_status (endometriosis_phenotype_status yes))
(patient_endo_symptoms (endometriosis yes) (abdominal_pelvic_pain yes) (dysmenorrhea yes) (pain_with_sex yes) (dyschezia yes) (dysuria yes) (infertility yes) (pelvic_perineal_pain yes) (adenomyosis yes))
(patient_concomitant_disease_symptoms (amenorrhea yes) (constipation yes) (diarrhea yes) (flank_pain yes) (hematuria yes) (frequent_urination yes) (ibs yes) (interstitial_cystitis yes))
Total facts: 5
